Modèle Logistic Regression (avec réduction de dimension avant PCA ou SVD)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.linear_model import LogisticRegression

# 1️⃣ Charger les datasets
train_df = pd.read_csv("application_train.csv")
test_df = pd.read_csv("application_test.csv")

# 2️⃣ Séparer features et cible
X_train = train_df.drop(columns=["TARGET", "SK_ID_CURR"])
y_train = train_df["TARGET"]
X_test = test_df.drop(columns=["SK_ID_CURR"])

# 3️⃣ Standardisation
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4️⃣ PCA (réduction de dimension)
pca = PCA(n_components=0.95)  # Garde 95% de la variance
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# 5️⃣ SVD (alternative pour sparse data)
svd = TruncatedSVD(n_components=50)  # On réduit à 50 dimensions
X_train_svd = svd.fit_transform(X_train_scaled)
X_test_svd = svd.transform(X_test_scaled)

# 6️⃣ Entraînement Logistic Regression (Baseline, PCA, SVD)
models = {
    "Baseline (Sans PCA/SVD)": X_train_scaled,
    "Avec PCA": X_train_pca,
    "Avec SVD": X_train_svd
}

submissions = {}

for name, X_train_model in models.items():
    X_test_model = X_test_pca if "PCA" in name else (X_test_svd if "SVD" in name else X_test_scaled)
    
    model = LogisticRegression(random_state=42, max_iter=1000)
    model.fit(X_train_model, y_train)
    
    y_pred_proba = model.predict_proba(X_test_model)[:, 1]
    
    # Stocker les prédictions
    submissions[name] = pd.DataFrame({"SK_ID_CURR": test_df["SK_ID_CURR"], "TARGET": y_pred_proba})
    submissions[name].to_csv(f"submission_{name.replace(' ', '_')}.csv", index=False)
    
    print(f"{name} - Prédictions sauvegardées.")



Modèle Random Forest

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

# 1️⃣ Charger les datasets
train_df = pd.read_csv("application_train.csv")
test_df = pd.read_csv("application_test.csv")

# 2️⃣ Séparer features et cible
X_train = train_df.drop(columns=["TARGET", "SK_ID_CURR"])
y_train = train_df["TARGET"]
X_test = test_df.drop(columns=["SK_ID_CURR"])

# 3️⃣ Entraînement du modèle Random Forest
rf_model = RandomForestClassifier(
    n_estimators=200,  # Nombre d'arbres (à ajuster)
    max_depth=10,  # Profondeur max (évite overfitting)
    random_state=42,
    n_jobs=-1,  # Utilise tous les CPU pour accélérer
    class_weight="balanced"  # Gère le déséquilibre des classes
)

rf_model.fit(X_train, y_train)

# 4️⃣ Prédictions sur application_test
y_pred_proba = rf_model.predict_proba(X_test)[:, 1]

# 5️⃣ Génération du fichier de soumission
submission_rf = pd.DataFrame({"SK_ID_CURR": test_df["SK_ID_CURR"], "TARGET": y_pred_proba})
submission_rf.to_csv("submission_RandomForest.csv", index=False)

print("✅ Random Forest - Prédictions sauvegardées.")


Modèle XGBOOST

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

# 1️⃣ Charger les datasets
train_df = pd.read_csv("application_train.csv")
test_df = pd.read_csv("application_test.csv")

# 2️⃣ Séparer features et cible
X_train = train_df.drop(columns=["TARGET", "SK_ID_CURR"])
y_train = train_df["TARGET"]
X_test = test_df.drop(columns=["SK_ID_CURR"])

# 3️⃣ Configuration de XGBoost
xgb_model = xgb.XGBClassifier(
    n_estimators=500,  # Nombre d'arbres
    learning_rate=0.05,  # Vitesse d'apprentissage
    max_depth=6,  # Profondeur des arbres
    subsample=0.8,  # Échantillonnage des données pour éviter l'overfitting
    colsample_bytree=0.8,  # Échantillonnage des features pour chaque arbre
    objective="binary:logistic",  # Problème de classification binaire
    eval_metric="auc",  # Optimisation du ROC-AUC
    use_label_encoder=False,
    random_state=42,
    n_jobs=-1  # Utilise tous les CPU
)

# 4️⃣ Entraînement du modèle
xgb_model.fit(X_train, y_train)

# 5️⃣ Prédictions sur application_test
y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]

# 6️⃣ Génération du fichier de soumission
submission_xgb = pd.DataFrame({"SK_ID_CURR": test_df["SK_ID_CURR"], "TARGET": y_pred_proba})
submission_xgb.to_csv("submission_XGBoost.csv", index=False)

print("✅ XGBoost - Prédictions sauvegardées.")


Modèle FNN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

# 1️⃣ Charger les datasets
train_df = pd.read_csv("application_train.csv")
test_df = pd.read_csv("application_test.csv")

# 2️⃣ Séparer features et labels
X_train = train_df.drop(columns=["TARGET", "SK_ID_CURR"])
y_train = train_df["TARGET"].values
X_test = test_df.drop(columns=["SK_ID_CURR"])

# 3️⃣ Normalisation (important pour les réseaux de neurones)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4️⃣ Conversion en Tensors PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)  # Reshape pour BCE Loss
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

# 5️⃣ Création du DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)

# 6️⃣ Définition du MLP (FNN)
class MLP(nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),  # Evite l’overfitting
            
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(64, 32),
            nn.ReLU(),
            
            nn.Linear(32, 1),
            nn.Sigmoid()  # Probabilité entre 0 et 1
        )

    def forward(self, x):
        return self.model(x)

# 7️⃣ Initialisation du modèle
input_dim = X_train.shape[1]
model = MLP(input_dim)

# 8️⃣ Définition de la Loss et de l’Optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 9️⃣ Entraînement du modèle
num_epochs = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# 🔟 Prédictions sur les données de test
model.eval()
with torch.no_grad():
    y_pred_proba = model(X_test_tensor.to(device)).cpu().numpy()

# 🔥 Sauvegarde du fichier de soumission
submission_nn = pd.DataFrame({"SK_ID_CURR": test_df["SK_ID_CURR"], "TARGET": y_pred_proba.flatten()})
submission_nn.to_csv("submission_NN.csv", index=False)

print("✅ Réseau de Neurones - Prédictions sauvegardées !")
